In [1]:
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import Dataset
import os
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        for filename in os.listdir(directory):
            if filename.endswith('.jpeg'):
                self.images.append(os.path.join(directory, filename))


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        if self.transform:
            image = self.transform(image)
        return image

# Use the transform while creating the dataset
transform = Compose([
    Resize((224, 224)),  # Resize the image to 224x224 pixels
    ToTensor(),  # Convert the image to PyTorch Tensor data type
])

dataset = CustomDataset('data/gtsrb/test_224', transform=transform)

KeyboardInterrupt: 

In [6]:
dataset[2].shape

torch.Size([3, 224, 224])

In [1]:
import numpy as np
import os
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
def compute_mean_std_per_channel(directories):
    num_images = 0
    sum_image = np.zeros(3)
    sum_squared_image = np.zeros(3)

    for directory in directories:
        for filename in os.listdir(directory):
            if filename.endswith('.jpeg') or filename.endswith('.JPEG'):
                image = Image.open(os.path.join(directory, filename))
                image = ToTensor()(image)
                for i in range(3):  # Loop over the 3 color channels
                    sum_image[i] += image[i, :, :].mean()
                    sum_squared_image[i] += image[i, :, :].std()
                num_images += 1

    mean = sum_image / num_images
    std = np.sqrt(sum_squared_image / num_images - mean ** 2)

    return mean, std

# Compute mean and std for the combined datasets
mean, std = compute_mean_std_per_channel(['data/imagenet/train'])

print(f'Combined dataset: mean = {mean}, std = {std}')

Combined dataset: mean = [nan nan nan], std = [nan nan nan]


/tmp/ipykernel_1443649/2205030506.py:20: RuntimeWarning: invalid value encountered in divide
  mean = sum_image / num_images
/tmp/ipykernel_1443649/2205030506.py:21: RuntimeWarning: invalid value encountered in divide
  std = np.sqrt(sum_squared_image / num_images - mean ** 2)


In [1]:
import os
import glob
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import torch
# 创建一个将图像转换为张量的转换
transform = transforms.ToTensor()

# 获取所有JPEG图像的路径
image_files = glob.glob(os.path.join('data/gtsrb/train', '*.[jJ][pP][eE][gG]'))

# 初始化列表以保存所有图像的张量
images = []

for image_file in image_files:
    # 打开图像并转换为张量
    image = Image.open(image_file)
    image_tensor = transform(image)
    images.append(image_tensor)

# 将所有图像张量堆叠到一个张量中
images = torch.stack(images)

# 计算mean和std
mean = images.mean([0, 2, 3])
std = images.std([0, 2, 3])

print('mean:', mean)
print('std:', std)

mean: tensor([0.3389, 0.3117, 0.3204])
std: tensor([0.2708, 0.2588, 0.2618])


In [2]:
import os
import random
from shutil import copyfile, rmtree

def select_quarter_images(directory, output_directory):
    # Create a dictionary to hold the images for each class
    class_images = {}

    # Loop over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.jpeg'):
            # Extract class from filename
            class_label = int(filename.split('_')[3].replace('.jpeg', '')[1:-1])
            # Add image to the list for this class
            class_images.setdefault(class_label, []).append(filename)

    # Select a quarter of the images for each class
    for class_label, images in class_images.items():
        selected_images = random.sample(images, len(images) // 5)
        # Copy selected images to output directory
        for image in selected_images:
            copyfile(os.path.join(directory, image), os.path.join(output_directory, image))

rmtree('data/gtsrb/selected_train_224', ignore_errors=True)
rmtree('data/stl10/selected_train_224', ignore_errors=True)
rmtree('data/svhn/selected_train_224', ignore_errors=True)

os.makedirs('data/gtsrb/selected_train_224', exist_ok=True)
os.makedirs('data/stl10/selected_train_224', exist_ok=True)
os.makedirs('data/svhn/selected_train_224', exist_ok=True)
# Select a quarter of the images from each class

select_quarter_images('data/gtsrb/train_224', 'data/gtsrb/selected_train_224')
select_quarter_images('data/stl10/train_224', 'data/stl10/selected_train_224')
select_quarter_images('data/svhn/train_224', 'data/svhn/selected_train_224')

In [6]:
import numpy as np
input=np.load('data/stl10/train_unlabeled.npz')
input['x'].shape

(105000, 32, 32, 3)

In [10]:
input['y'].shape

(5000, 1)

In [8]:
input=np.load('data/stl10/train.npz')
input['x'].shape

(5000, 32, 32, 3)

In [6]:
import torch
net = torch.load("benchmark_logs/cifar10/version_1/SimSiam/checkpoints/epoch=0-step=24.ckpt")
net

{'epoch': 0,
 'global_step': 24,
 'pytorch-lightning_version': '2.2.4',
 'state_dict': OrderedDict([('backbone.0.weight',
               tensor([[[[-0.0008,  0.1025, -0.1586],
                         [-0.1401, -0.0741,  0.0495],
                         [-0.0034,  0.1509, -0.0197]],
               
                        [[ 0.0515, -0.0579, -0.0385],
                         [-0.1820, -0.1271, -0.0807],
                         [ 0.0077,  0.0749,  0.1123]],
               
                        [[-0.1256, -0.0798,  0.0726],
                         [ 0.1640, -0.0356,  0.1457],
                         [-0.0261,  0.0237,  0.1749]]],
               
               
                       [[[-0.1775, -0.1205, -0.0489],
                         [-0.0738,  0.1667, -0.1234],
                         [-0.0870, -0.1321, -0.1778]],
               
                        [[-0.1102,  0.1661,  0.0865],
                         [ 0.0950,  0.0126, -0.0959],
                         [ 0.0349, -0

In [5]:
data = torch.randn(8, 3, 32, 32)
net(data).flatten(start_dim=1).shape

torch.Size([8, 512])

In [1]:
import torch
net = torch.load("output/MOCO/gtsrb_backdoored_encoder/2024-05-13-23:22:46/model_160.pth")
net

{'epoch': 160,
 'state_dict': OrderedDict([('0.weight',
               tensor([[[[ 0.0435, -0.0668, -0.0591],
                         [ 0.1285, -0.1624,  0.0702],
                         [-0.0619,  0.0900,  0.0515]],
               
                        [[ 0.0606,  0.0395, -0.0218],
                         [ 0.1007, -0.1215,  0.0021],
                         [-0.0571,  0.0053,  0.0241]],
               
                        [[ 0.1049,  0.0257, -0.0671],
                         [ 0.0382, -0.1005, -0.0449],
                         [-0.0554,  0.0247,  0.0734]]],
               
               
                       [[[ 0.0563, -0.0440,  0.0593],
                         [ 0.0532,  0.1156,  0.0395],
                         [-0.0709, -0.0774, -0.0568]],
               
                        [[ 0.0993,  0.0101,  0.0233],
                         [ 0.0313,  0.1049, -0.0690],
                         [ 0.0748, -0.0058, -0.0824]],
               
                        [[ 0.075